In [ ]:
import tensorflow as tf
import collections
import numpy as np
import tensorflow_federated as tff
from tqdm import tqdm
import pandas as pd
import keras_tuner as kt
from sklearn.model_selection import train_test_split

import import_ipynb

In [ ]:
class ModelHelper:

    def __init__(self, df, total_window_length=None):
        self.df = df
        if total_window_length is not None:
            self.sequence_length = total_window_length - 1
            self.total_window_length = total_window_length
        self.target_column_name = df.columns[0]
        self.num_epochs = 20

    def reset_df(self, df):
        self.df = df

    def train_val_test_split(self):
        train, self.df_test = train_test_split(self.df, test_size=0.2, shuffle=False)
        self.df_train, self.df_val = train_test_split(train, test_size=0.2, shuffle=False)

    def set_column_names(self, column_names):
        self.column_names = column_names

    def set_numerical_column_names(self, numerical_column_names):
        self.numerical_column_names = numerical_column_names

    def set_target_column_name(self, target_column_name):
        self.target_column_name = target_column_name

    def set_vocab_size(self, vocab_size=None):
        if vocab_size is None:
            self.vocab_size = int(np.max(self.df[self.target_column_name].values) + 1)
        else:
            self.vocab_size = vocab_size

    def set_client_column_name(self, column_name):
        self.client_column = column_name

    def set_client_column_ids(self, ids=None):
        if ids is None:
            self.client_column_ids = self.df[self.client_column].unique()
        else:
            self.client_column_ids = ids

    def set_batch_size(self, batch_size):
        self.batch_size = batch_size

    def set_num_epochs(self, num_epochs):
        self.num_epochs = num_epochs

    def set_prefetch_buffer(self, prefetch_buffer):
        self.prefetch_buffer = prefetch_buffer

    def set_hyper_parameters(self, num_epochs, batch_size, prefetch_buffer):
        self.num_epochs = num_epochs
        self.batch_size = batch_size
        self.prefetch_buffer = prefetch_buffer

    def get_unique_clients(self):
        if self.client_column_ids is None and self.client_column in self.df:
            return self.df[self.client_column].unique()
        return self.client_column_ids


For simple model

In [ ]:
def df_to_dataset(self, df, shuffle=False):

  df = df.copy()
  labels = df.pop(self.target_column_name)
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df))
  ds = ds.batch(self.batch_size)
  return ds

In [ ]:
def df_to_datasets(self, shuffle=False):

    self.train_dataset = df_to_dataset(self, self.df_train, shuffle)
    self.val_dataset = df_to_dataset(self, self.df_val, shuffle)
    self.test_dataset = df_to_dataset(self, self.df_test, shuffle)

ModelHelper.df_to_datasets = df_to_datasets

Split data for FL

In [ ]:
def split_data_users(self):

  # dictionary of list of df
  df_dictionary = {}

  for uid in tqdm(self.client_column_ids):
    # Get the records of the user
    user_df_train = self.df_train.loc[self.df_train[self.client_column] == uid].copy()
    user_df_val = self.df_val.loc[self.df_val[self.client_column] == uid].copy()
    user_df_test = self.df_test.loc[self.df_test[self.client_column] == uid].copy()

    # Get a list of dataframes of length N records
    user_list_train = [user_df_train[i:i+self.total_window_length] for i in range(0, user_df_train.shape[0], self.total_window_length)]
    user_list_val = [user_df_val[i:i+self.total_window_length] for i in range(0, user_df_val.shape[0], self.total_window_length)]
    user_list_test = [user_df_test[i:i+self.total_window_length] for i in range(0, user_df_test.shape[0], self.total_window_length)]

    # Save the list of dataframes into a dictionary
    df_dictionary[uid] = {
        'train': user_list_train,
        'val': user_list_val,
        'test': user_list_test
    }

  self.df_dictionary=df_dictionary

ModelHelper.split_data_users = split_data_users

Split Data Basic

In [ ]:
# Split the data into chunks of total_window_length
def split_data(self):

  # Get a list of dataframes of length total_window_length records
  list_train = ([self.df_train[i:i+self.total_window_length] for i in range(0, self.df_train.shape[0], self.total_window_length)])
  list_val = ([self.df_val[i:i+self.total_window_length] for i in range(0, self.df_val.shape[0], self.total_window_length)])
  list_test = ([self.df_test[i:i+self.total_window_length] for i in range(0, self.df_test.shape[0], self.total_window_length)])

  self.list_train=list_train
  self.list_val=list_val
  self.list_test=list_test

ModelHelper.split_data = split_data

Sliding Window Approach

In [ ]:
# Split the data into chunks of N
def split_data_sliding(self):
    self.list_train = sliding_window(self.total_window_length, self.df_train)
    self.list_val = sliding_window(self.total_window_length, self.df_val)
    self.list_test = sliding_window(self.total_window_length, self.df_test)

ModelHelper.split_data_sliding = split_data_sliding

In [ ]:
def sliding_window(N, arr):

    """
    Splits an array into a list of subarrays using a sliding window approach.

    Parameters:
        arr: a numpy array
        N: the number of elements in each subarray

    Returns:
        A list of numpy arrays.
    """
    # Get the number of subarrays
    num_subarrays = arr.shape[0] - N + 1

    # Create an empty list to store the subarrays
    subarrays = []

    # Loop through the array and create the subarrays
    for i in range(num_subarrays):
        subarray = arr[i:i + N]
        subarrays.append(subarray)

    return subarrays

Create TF DataSet for Basic (Non-Federated)

In [ ]:
# Takes a dictionary with train, validation and test sets and the desired set type
def create_dataset(self, my_list, multi_target=True):

  input_dict = collections.OrderedDict()

  # If the last dataframe of the list is not complete
  if len(my_list[-1]) < self.total_window_length:
    diff = 1
  else:
    diff = 0

  if len(my_list) > 0:
    # Create the dictionary to create a clientData
    for header in self.column_names:
      input_dict[header] = [my_list[i][header].values[:-1] for i in range(0, len(my_list) - diff)]

  if multi_target is True:
    dataset = tf.data.Dataset.from_tensor_slices((input_dict, np.array([my_list[i][self.column_names[0]].values[1:] for i in range(0, len(my_list)-diff)])))
  else:
    dataset = tf.data.Dataset.from_tensor_slices((input_dict, np.array([my_list[i][self.column_names[0]].values[-1] for i in range(0, len(my_list)-diff)])))


  return dataset

ModelHelper.create_dataset = create_dataset

In [ ]:
def create_and_batch_datasets(self, multi_target=True):
    self.train_dataset = self.create_dataset(self.list_train, multi_target).batch(self.batch_size, drop_remainder=True)
    self.val_dataset = self.create_dataset(self.list_val, multi_target).batch(self.batch_size, drop_remainder=True)
    self.test_dataset = self.create_dataset(self.list_test, multi_target).batch(self.batch_size, drop_remainder=True)

ModelHelper.create_and_batch_datasets = create_and_batch_datasets

Create Federated Learning Client Dictionaries

In [ ]:
# Takes a dictionary with train, validation and test sets and the desired set type
# for users for federated training
def create_clients_dict(self, set_type):

  dataset_dict = {}

  for uid in tqdm(self.client_column_ids):

    c_data = collections.OrderedDict()
    values = self.df_dictionary[uid][set_type]

    # If the last dataframe of the list is not complete #
    if len(values[-1]) < self.total_window_length:
      diff = 1
    else:
      diff = 0

    if len(values) > 0:
      # Create the dictionary to create a clientData
      for header in self.column_names:
        c_data[header] = [values[i][header].values for i in range(0, len(values)-diff)]
      dataset_dict[uid] = c_data

  return dataset_dict

In [ ]:
def generate_client_dicts(self):

    self.clients_train_dict = create_clients_dict(self, 'train')
    self.clients_val_dict = create_clients_dict(self, 'val')
    self.clients_test_dict = create_clients_dict(self, 'test')

ModelHelper.generate_client_dicts = generate_client_dicts

FL Preprocessing routine

In [ ]:
# preprocess dataset to tf format
def preprocess(self, dataset, N):

  def batch_format_fn(element):

    x=collections.OrderedDict()

    for name in self.column_names:
      x[name]=tf.reshape(element[name][:, :-1], [-1, N-1])

    y=tf.reshape(element[self.column_names[0]][:, 1:], [-1, N-1])

    return collections.OrderedDict(x=x, y=y)

  return dataset.repeat(self.num_epochs).batch(self.batch_size, drop_remainder=True).map(batch_format_fn).prefetch(self.prefetch_buffer)

Create TF slices Client Data

In [ ]:
def generate_client_data(self):

    self.client_train_data = tff.simulation.FromTensorSlicesClientData(self.clients_train_dict)
    self.client_val_data = tff.simulation.FromTensorSlicesClientData(self.clients_val_dict)
    self.client_test_data = tff.simulation.FromTensorSlicesClientData(self.clients_test_dict)

ModelHelper.generate_client_data = generate_client_data

Create Federated Data for clients

In [ ]:
# create federated data for every client
def make_federated_data(self, client_data, client_ids):

  return [
      preprocess(self, client_data.create_tf_dataset_for_client(x), self.total_window_length)
      for x in tqdm(client_ids)
  ]

In [ ]:
def generate_federated_data(self, sample_clients):

    # Federate the clients datasets
    self.federated_train_data = make_federated_data(self, self.client_train_data, sample_clients)
    self.federated_val_data = make_federated_data(self, self.client_val_data, sample_clients)
    self.federated_test_data = make_federated_data(self, self.client_test_data, sample_clients)

ModelHelper.generate_federated_data = generate_federated_data

NYC Central (Non-Federated) Model Functions

In [ ]:
# function to remove duplicates
def create_sequence(locations):

  # Flatten the list of places
  sequence = np.reshape(locations.values, [-1])

  # Create a temporary array of the same lenght of the sequece of locations
  copy = np.zeros(sequence.shape[0], dtype=np.int32)

  # Copy the sequence of location in the copy array but shifted right by 1 position
  # The last location does not need to be copied, it can't be a duplicate
  copy[1:] = sequence[:sequence.shape[0]-1]

  # Where we get 0 it can be a possible duplicated
  duplicated = sequence - copy

  # indices where the subtraction gives 0
  idx = np.where(duplicated == 0)[0]

  # Find where the position of the zeros are even
  even = idx%2 == 0

  # List the indices where the position is even and the subtraction gave 0
  to_drop = idx[even]

  # Remove the duplicates
  clean_sequence = np.delete(sequence, to_drop)
  return clean_sequence, to_drop

In [ ]:
def df_to_location_sequence(self):

  # take just the columns we need
  locations = self.df[['pickup_location_id','dropoff_location_id']].copy()
  locations = locations.astype('int32')


  # define the indices to keep trace of the locations
  x = np.arange(0, locations.values.shape[0])

  pos = np.array([x,x]).T
  pos = np.reshape(pos, [-1])

  # Represent whether the location is a pickup or a dropoff
  pick = np.zeros(locations.values.shape[0], dtype=int)
  drop = np.ones(locations.values.shape[0], dtype=int)

  loc = np.array([pick,drop]).T
  loc = np.reshape(loc, [-1])

  # Generate the sequence of places
  sequence, duplicates = create_sequence(locations)

  # We use now the indices of the duplicated locations to clean also the array of rows and the array of location types
  pos = np.delete(pos, duplicates)
  loc = np.delete(loc, duplicates)

  # Select the indices of records we want the pickup location
  pick_pos = pos[pos[loc == 0]]

  # Select the indices of records we want the dropoff location
  drop_pos = pos[pos[loc == 1]]


  records_pick = self.df.iloc[pick_pos][['pickup_location_id', 'pickup_week_day',	'pickup_hour',	'pickup_day',	'pickup_month']]
  records_pick = records_pick.rename(columns={'pickup_location_id': 'location_id', 'pickup_week_day':'week_day' ,	'pickup_hour':'hour' ,	'pickup_day':	'day' ,	'pickup_month':'month' })

  idx_drop = np.nonzero(loc == 0)[0]
  records_drop = self.df.iloc[drop_pos][['dropoff_location_id', 'dropoff_week_day',	'dropoff_hour',	'dropoff_day',	'dropoff_month']]
  records_drop = records_drop.rename(columns={'dropoff_location_id': 'location_id', 'dropoff_week_day':'week_day' ,	'dropoff_hour':'hour' ,	'dropoff_day':	'day' ,	'dropoff_month':'month' })

  locations_sequence = pd.concat([records_pick, records_drop])

  # reset the index
  locations_sequence.reset_index(inplace=True)

  # From hour to sin-cos representation
  locations_sequence['hour_sin'] = np.sin(locations_sequence.hour*(2.*np.pi/24))
  locations_sequence['hour_cos'] = np.cos(locations_sequence.hour*(2.*np.pi/24))

  locations_sequence['week_day_sin'] = np.sin(locations_sequence.week_day*(2.*np.pi/7))
  locations_sequence['week_day_cos'] = np.cos(locations_sequence.week_day*(2.*np.pi/7))


  # Drop the original column
  locations_sequence.drop(['hour'], axis=1, inplace=True)


  # Helper function to encode the day_type
  def is_weekend(days):
    weekends = np.zeros(len(days))
    weekends[((days == 5) | (days == 6))] = 1
    return weekends

  # Apply the helper function to all the records
  locations_sequence['weekend'] = is_weekend(locations_sequence['week_day'])

  # the column is not needed anymore
  locations_sequence.drop(['week_day'], axis=1, inplace=True)

  # Correct the weekend feature type
  dictionary = {'weekend': 'int32'}
  locations_sequence = locations_sequence.astype(dictionary, copy=True)
  self.df = locations_sequence

  return locations_sequence, pos, loc

ModelHelper.df_to_location_sequence = df_to_location_sequence

create batch dataset for basic (non-federated) execution

In [ ]:
def create_batch_dataset(self):

    if len(self.list_train[-1]) < self.total_window_length:
      diff_train = 1
    else:
      diff_train = 0

    if len(self.list_val[-1]) < self.total_window_length:
      diff_val = 1
    else:
      diff_val = 0

    if len(self.list_test[-1]) < self.total_window_length:
      diff_test = 1
    else:
      diff_test = 0

    # Define the input features of the  dataset
    train_input_dict = {
      'start_place':np.array([self.list_train[i]['location_id'].values[:-1] for i in range(0, len(self.list_train)-diff_train)]),
      'start_hour_sin':np.array([self.list_train[i]['hour_sin'].values[:-1] for i in range(0, len(self.list_train)-diff_train)]),
      'start_hour_cos':np.array([self.list_train[i]['hour_cos'].values[:-1] for i in range(0, len(self.list_train)-diff_train)]),
      'weekend':np.array([self.list_train[i]['weekend'].values[:-1] for i in range(0, len(self.list_train)-diff_train)]),
      'week_day_sin':np.array([self.list_train[i]['week_day_sin'].values[:-1] for i in range(0, len(self.list_train)-diff_train)]),
      'week_day_cos':np.array([self.list_train[i]['week_day_cos'].values[:-1] for i in range(0, len(self.list_train)-diff_train)]),
      'end_hour_sin':np.array([self.list_train[i]['hour_sin'].values[-1] for i in range(0, len(self.list_train)-diff_train)]),
      'end_hour_cos':np.array([self.list_train[i]['hour_cos'].values[-1] for i in range(0, len(self.list_train)-diff_train)]),
      'end_weekend':np.array([self.list_train[i]['weekend'].values[-1] for i in range(0, len(self.list_train)-diff_train)]),
      'end_week_day_sin':np.array([self.list_train[i]['week_day_sin'].values[-1] for i in range(0, len(self.list_train)-diff_train)]),
      'end_week_day_cos':np.array([self.list_train[i]['week_day_cos'].values[-1] for i in range(0, len(self.list_train)-diff_train)]),
    }

    # Define the input features of the  dataset
    val_input_dict = {
      'start_place':np.array([self.list_val[i]['location_id'].values[:-1] for i in range(0, len(self.list_val)-diff_val)]),
      'start_hour_sin':np.array([self.list_val[i]['hour_sin'].values[:-1] for i in range(0, len(self.list_val)-diff_val)]),
      'start_hour_cos':np.array([self.list_val[i]['hour_cos'].values[:-1] for i in range(0, len(self.list_val)-diff_val)]),
      'weekend':np.array([self.list_val[i]['weekend'].values[:-1] for i in range(0, len(self.list_val)-diff_val)]),
      'week_day_sin':np.array([self.list_val[i]['week_day_sin'].values[:-1] for i in range(0, len(self.list_val)-diff_val)]),
      'week_day_cos':np.array([self.list_val[i]['week_day_cos'].values[:-1] for i in range(0, len(self.list_val)-diff_val)]),
      'end_hour_sin':np.array([self.list_val[i]['hour_sin'].values[-1] for i in range(0, len(self.list_val)-diff_val)]),
      'end_hour_cos':np.array([self.list_val[i]['hour_cos'].values[-1] for i in range(0, len(self.list_val)-diff_val)]),
      'end_weekend':np.array([self.list_val[i]['weekend'].values[-1] for i in range(0, len(self.list_val)-diff_val)]),
      'end_week_day_sin':np.array([self.list_val[i]['week_day_sin'].values[-1] for i in range(0, len(self.list_val)-diff_val)]),
      'end_week_day_cos':np.array([self.list_val[i]['week_day_cos'].values[-1] for i in range(0, len(self.list_val)-diff_val)]),
    }

    # Define the input features of the  dataset
    test_input_dict = {
      'start_place':np.array([self.list_test[i]['location_id'].values[:-1] for i in range(0, len(self.list_test)-diff_test)]),
      'start_hour_sin':np.array([self.list_test[i]['hour_sin'].values[:-1] for i in range(0, len(self.list_test)-diff_test)]),
      'start_hour_cos':np.array([self.list_test[i]['hour_cos'].values[:-1] for i in range(0, len(self.list_test)-diff_test)]),
      'weekend':np.array([self.list_test[i]['weekend'].values[:-1] for i in range(0, len(self.list_test)-diff_test)]),
      'week_day_sin':np.array([self.list_test[i]['week_day_sin'].values[:-1] for i in range(0, len(self.list_test)-diff_test)]),
      'week_day_cos':np.array([self.list_test[i]['week_day_cos'].values[:-1] for i in range(0, len(self.list_test)-diff_test)]),
      'end_hour_sin':np.array([self.list_test[i]['hour_sin'].values[-1] for i in range(0, len(self.list_test)-diff_test)]),
      'end_hour_cos':np.array([self.list_test[i]['hour_cos'].values[-1] for i in range(0, len(self.list_test)-diff_test)]),
      'end_weekend':np.array([self.list_test[i]['weekend'].values[-1] for i in range(0, len(self.list_test)-diff_test)]),
      'end_week_day_sin':np.array([self.list_test[i]['week_day_sin'].values[-1] for i in range(0, len(self.list_test)-diff_test)]),
      'end_week_day_cos':np.array([self.list_test[i]['week_day_cos'].values[-1] for i in range(0, len(self.list_test)-diff_test)]),
    }

    # Create training examples / targets, we are going to predict the next location
    trips_dataset_train = tf.data.Dataset.from_tensor_slices((train_input_dict, np.array([self.list_train[i]['location_id'].values[-1] for i in range(0, len(self.list_train)-diff_train)])))
    trips_dataset_val = tf.data.Dataset.from_tensor_slices((val_input_dict, np.array([self.list_val[i]['location_id'].values[-1] for i in range(0, len(self.list_val)-diff_val)])))
    trips_dataset_test = tf.data.Dataset.from_tensor_slices((test_input_dict, np.array([self.list_test[i]['location_id'].values[-1] for i in range(0, len(self.list_test)-diff_test)])))


    # Buffer size to shuffle the dataset
    # (TF data is designed to work with possibly infinite sequences,
    # so it doesn't attempt to shuffle the entire sequence in memory. Instead,
    # it maintains a buffer in which it shuffles elements).
    # BUFFER_SIZE = 10000

    # Create the dataset by creating batches
    # Uncomment the shuffle function in case we want to shuffle the sequences
    self.train_dataset = trips_dataset_train.batch(self.batch_size, drop_remainder=True) #.shuffle(BUFFER_SIZE)
    self.val_dataset = trips_dataset_val.batch(self.batch_size, drop_remainder=True) #.shuffle(BUFFER_SIZE)
    self.test_dataset = trips_dataset_test.batch(self.batch_size, drop_remainder=True) #.shuffle(BUFFER_SIZE)

    return self.train_dataset, self.val_dataset, self.test_dataset

ModelHelper.create_batch_dataset = create_batch_dataset

In [ ]:
def create_batch_dataset_cin(self):

    if len(self.list_train[-1]) < self.total_window_length:
      diff_train = 1
    else:
      diff_train = 0

    if len(self.list_val[-1]) < self.total_window_length:
      diff_val = 1
    else:
      diff_val = 0

    if len(self.list_test[-1]) < self.total_window_length:
      diff_test = 1
    else:
      diff_test = 0

    # Define the input features of the  dataset
    train_input_dict = {
      'start_place':np.array([self.list_train[i]['location_id'].values[:-1] for i in range(0, len(self.list_train)-diff_train)]),
      'start_hour_sin':np.array([self.list_train[i]['clock_sin'].values[:-1] for i in range(0, len(self.list_train)-diff_train)]),
      'start_hour_cos':np.array([self.list_train[i]['clock_cos'].values[:-1] for i in range(0, len(self.list_train)-diff_train)]),
      'weekend':np.array([self.list_train[i]['is_weekend'].values[:-1] for i in range(0, len(self.list_train)-diff_train)]),
      'week_day_sin':np.array([self.list_train[i]['week_day_sin'].values[:-1] for i in range(0, len(self.list_train)-diff_train)]),
      'week_day_cos':np.array([self.list_train[i]['week_day_cos'].values[:-1] for i in range(0, len(self.list_train)-diff_train)]),
      'end_hour_sin':np.array([self.list_train[i]['clock_sin'].values[-1] for i in range(0, len(self.list_train)-diff_train)]),
      'end_hour_cos':np.array([self.list_train[i]['clock_cos'].values[-1] for i in range(0, len(self.list_train)-diff_train)]),
      'end_weekend':np.array([self.list_train[i]['is_weekend'].values[-1] for i in range(0, len(self.list_train)-diff_train)]),
      'end_week_day_sin':np.array([self.list_train[i]['week_day_sin'].values[-1] for i in range(0, len(self.list_train)-diff_train)]),
      'end_week_day_cos':np.array([self.list_train[i]['week_day_cos'].values[-1] for i in range(0, len(self.list_train)-diff_train)]),
    }

    # Define the input features of the  dataset
    val_input_dict = {
      'start_place':np.array([self.list_val[i]['location_id'].values[:-1] for i in range(0, len(self.list_val)-diff_val)]),
      'start_hour_sin':np.array([self.list_val[i]['clock_sin'].values[:-1] for i in range(0, len(self.list_val)-diff_val)]),
      'start_hour_cos':np.array([self.list_val[i]['clock_cos'].values[:-1] for i in range(0, len(self.list_val)-diff_val)]),
      'weekend':np.array([self.list_val[i]['is_weekend'].values[:-1] for i in range(0, len(self.list_val)-diff_val)]),
      'week_day_sin':np.array([self.list_val[i]['week_day_sin'].values[:-1] for i in range(0, len(self.list_val)-diff_val)]),
      'week_day_cos':np.array([self.list_val[i]['week_day_cos'].values[:-1] for i in range(0, len(self.list_val)-diff_val)]),
      'end_hour_sin':np.array([self.list_val[i]['clock_sin'].values[-1] for i in range(0, len(self.list_val)-diff_val)]),
      'end_hour_cos':np.array([self.list_val[i]['clock_cos'].values[-1] for i in range(0, len(self.list_val)-diff_val)]),
      'end_weekend':np.array([self.list_val[i]['is_weekend'].values[-1] for i in range(0, len(self.list_val)-diff_val)]),
      'end_week_day_sin':np.array([self.list_val[i]['week_day_sin'].values[-1] for i in range(0, len(self.list_val)-diff_val)]),
      'end_week_day_cos':np.array([self.list_val[i]['week_day_cos'].values[-1] for i in range(0, len(self.list_val)-diff_val)]),
    }

    # Define the input features of the  dataset
    test_input_dict = {
      'start_place':np.array([self.list_test[i]['location_id'].values[:-1] for i in range(0, len(self.list_test)-diff_test)]),
      'start_hour_sin':np.array([self.list_test[i]['clock_sin'].values[:-1] for i in range(0, len(self.list_test)-diff_test)]),
      'start_hour_cos':np.array([self.list_test[i]['clock_cos'].values[:-1] for i in range(0, len(self.list_test)-diff_test)]),
      'weekend':np.array([self.list_test[i]['is_weekend'].values[:-1] for i in range(0, len(self.list_test)-diff_test)]),
      'week_day_sin':np.array([self.list_test[i]['week_day_sin'].values[:-1] for i in range(0, len(self.list_test)-diff_test)]),
      'week_day_cos':np.array([self.list_test[i]['week_day_cos'].values[:-1] for i in range(0, len(self.list_test)-diff_test)]),
      'end_hour_sin':np.array([self.list_test[i]['clock_sin'].values[-1] for i in range(0, len(self.list_test)-diff_test)]),
      'end_hour_cos':np.array([self.list_test[i]['clock_cos'].values[-1] for i in range(0, len(self.list_test)-diff_test)]),
      'end_weekend':np.array([self.list_test[i]['is_weekend'].values[-1] for i in range(0, len(self.list_test)-diff_test)]),
      'end_week_day_sin':np.array([self.list_test[i]['week_day_sin'].values[-1] for i in range(0, len(self.list_test)-diff_test)]),
      'end_week_day_cos':np.array([self.list_test[i]['week_day_cos'].values[-1] for i in range(0, len(self.list_test)-diff_test)]),
    }

    # Create training examples / targets, we are going to predict the next location
    trips_dataset_train = tf.data.Dataset.from_tensor_slices((train_input_dict, np.array([self.list_train[i]['location_id'].values[-1] for i in range(0, len(self.list_train)-diff_train)])))
    trips_dataset_val = tf.data.Dataset.from_tensor_slices((val_input_dict, np.array([self.list_val[i]['location_id'].values[-1] for i in range(0, len(self.list_val)-diff_val)])))
    trips_dataset_test = tf.data.Dataset.from_tensor_slices((test_input_dict, np.array([self.list_test[i]['location_id'].values[-1] for i in range(0, len(self.list_test)-diff_test)])))


    # Buffer size to shuffle the dataset
    # (TF data is designed to work with possibly infinite sequences,
    # so it doesn't attempt to shuffle the entire sequence in memory. Instead,
    # it maintains a buffer in which it shuffles elements).
    # BUFFER_SIZE = 10000

    # Create the dataset by creating batches
    # Uncomment the shuffle function in case we want to shuffle the sequences
    self.train_dataset = trips_dataset_train.batch(self.batch_size, drop_remainder=True) #.shuffle(BUFFER_SIZE)
    self.val_dataset = trips_dataset_val.batch(self.batch_size, drop_remainder=True) #.shuffle(BUFFER_SIZE)
    self.test_dataset = trips_dataset_test.batch(self.batch_size, drop_remainder=True) #.shuffle(BUFFER_SIZE)

    return self.train_dataset, self.val_dataset, self.test_dataset

ModelHelper.create_batch_dataset_cin = create_batch_dataset_cin

Window Generator Approach

modified and manually imported "timeseries_dataset_from_array"

In [ ]:

# IMPORTANT CHANGE : "drop_remainder" SET TO True SO THAT THE DATASET IS ALWAYS THE RIGHT SIZE
# (THIS GUARANTEES HAVING NO INCOMPLETE SHAPES IN LAST THE BATCH)
# dataset = dataset.batch(batch_size, drop_remainder=True)

def timeseries_dataset_from_array(
    data,
    targets,
    sequence_length,
    sequence_stride=1,
    sampling_rate=1,
    batch_size=128,
    shuffle=False,
    seed=None,
    start_index=None,
    end_index=None,
):
    if start_index:
        if start_index < 0:
            raise ValueError(
                "`start_index` must be 0 or greater. Received: "
                f"start_index={start_index}"
            )
        if start_index >= len(data):
            raise ValueError(
                "`start_index` must be lower than the length of the "
                f"data. Received: start_index={start_index}, for data "
                f"of length {len(data)}"
            )
    if end_index:
        if start_index and end_index <= start_index:
            raise ValueError(
                "`end_index` must be higher than `start_index`. "
                f"Received: start_index={start_index}, and "
                f"end_index={end_index} "
            )
        if end_index >= len(data):
            raise ValueError(
                "`end_index` must be lower than the length of the "
                f"data. Received: end_index={end_index}, for data of "
                f"length {len(data)}"
            )
        if end_index <= 0:
            raise ValueError(
                "`end_index` must be higher than 0. "
                f"Received: end_index={end_index}"
            )

    # Validate strides
    if sampling_rate <= 0:
        raise ValueError(
            "`sampling_rate` must be higher than 0. Received: "
            f"sampling_rate={sampling_rate}"
        )
    if sampling_rate >= len(data):
        raise ValueError(
            "`sampling_rate` must be lower than the length of the "
            f"data. Received: sampling_rate={sampling_rate}, for data "
            f"of length {len(data)}"
        )
    if sequence_stride <= 0:
        raise ValueError(
            "`sequence_stride` must be higher than 0. Received: "
            f"sequence_stride={sequence_stride}"
        )
    if sequence_stride >= len(data):
        raise ValueError(
            "`sequence_stride` must be lower than the length of the "
            f"data. Received: sequence_stride={sequence_stride}, for "
            f"data of length {len(data)}"
        )

    if start_index is None:
        start_index = 0
    if end_index is None:
        end_index = len(data)

    # Determine the lowest dtype to store start positions (to lower memory
    # usage).
    num_seqs = end_index - start_index - (sequence_length * sampling_rate) + 1
    if targets is not None:
        num_seqs = min(num_seqs, len(targets))
    if num_seqs < 2147483647:
        index_dtype = "int32"
    else:
        index_dtype = "int64"

    # Generate start positions
    start_positions = np.arange(0, num_seqs, sequence_stride, dtype=index_dtype)
    if shuffle:
        if seed is None:
            seed = np.random.randint(1e6)
        rng = np.random.RandomState(seed)
        rng.shuffle(start_positions)

    sequence_length = tf.cast(sequence_length, dtype=index_dtype)
    sampling_rate = tf.cast(sampling_rate, dtype=index_dtype)

    positions_ds = tf.data.Dataset.from_tensors(start_positions).repeat()

    # For each initial window position, generates indices of the window elements
    indices = tf.data.Dataset.zip(
        (tf.data.Dataset.range(len(start_positions)), positions_ds)
    ).map(
        lambda i, positions: tf.range(
            positions[i],
            positions[i] + sequence_length * sampling_rate,
            sampling_rate,
        ),
        num_parallel_calls=tf.data.experimental.AUTOTUNE,
    )

    dataset = sequences_from_indices(data, indices, start_index, end_index)
    if targets is not None:
        indices = tf.data.Dataset.zip(
            (tf.data.Dataset.range(len(start_positions)), positions_ds)
        ).map(
            lambda i, positions: positions[i],
            num_parallel_calls=tf.data.experimental.AUTOTUNE,
        )
        target_ds = sequences_from_indices(
            targets, indices, start_index, end_index
        )
        dataset = tf.data.Dataset.zip((dataset, target_ds))
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    if batch_size is not None:
        if shuffle:
            # Shuffle locally at each iteration
            dataset = dataset.shuffle(buffer_size=batch_size * 8, seed=seed)
        dataset = dataset.batch(batch_size, drop_remainder=True)
    else:
        if shuffle:
            dataset = dataset.shuffle(buffer_size=1024, seed=seed)
    return dataset

In [ ]:
def sequences_from_indices(array, indices_ds, start_index, end_index):

    dataset = tf.data.Dataset.from_tensors(array[start_index:end_index])
    dataset = tf.data.Dataset.zip((dataset.repeat(), indices_ds)).map(
        lambda steps, inds: tf.gather(steps, inds),
        num_parallel_calls=tf.data.experimental.AUTOTUNE,
    )
    return dataset

In [ ]:
def create_users_locations(self):

    users_locations = []

    # For each user
    df = self.df
    for c_id in tqdm(self.get_unique_clients()):
      # Call the function
      self.reset_df(df.loc[df[self.client_column] == c_id].copy())
      locations_sequence, pos, loc = self.df_to_location_sequence()
      # Add the sequence df of the user to the list
      users_locations.append(locations_sequence)

    self.users_locations = users_locations
    return users_locations

ModelHelper.create_users_locations = create_users_locations

In [ ]:
def split_concat_user_df(self):

    users_locations_train = []
    users_locations_val = []
    users_locations_test = []

    for user_df in self.users_locations:
        # Split in train, test and validation
        train, test = train_test_split(user_df, test_size=0.2, shuffle=False)
        train, val = train_test_split(train, test_size=0.2, shuffle=False)

        # Append the sets
        users_locations_train.append(train)
        users_locations_val.append(val)
        users_locations_test.append(test)

    df_train = pd.concat(users_locations_train)
    df_train.drop(['index', 'day', 'month'], axis=1, inplace=True)
    df_train['location_id']  = pd.to_numeric(df_train['location_id'], downcast='integer')
    self.df_train = df_train

    # Merge back the dataframes
    df_val = pd.concat(users_locations_val)
    df_val.drop(['index', 'day', 'month'], axis=1, inplace=True)
    df_val['location_id']  = pd.to_numeric(df_val['location_id'], downcast='integer')
    self.df_val = df_train

    # Merge back the dataframes
    df_test = pd.concat(users_locations_test)
    df_test.drop(['index', 'day', 'month'], axis=1, inplace=True)
    df_test['location_id']  = pd.to_numeric(df_test['location_id'], downcast='integer')
    self.df_test = df_train

ModelHelper.split_concat_user_df = split_concat_user_df

In [ ]:
def basic_split_df(self):

    self.df_train, self.df_test =  train_test_split(self.df, test_size=0.2, shuffle=False)
    self.df_train, self.df_val = train_test_split(self.df_train, test_size=0.2, shuffle=False)

ModelHelper.basic_split_df = basic_split_df

In [ ]:
def drop_all_but_target(self):

    columns = list(set(self.df_train.columns.values) - set([self.target_column_name]))

    self.df_train.drop(columns, axis=1, inplace=True, errors='ignore')
    self.df_val.drop(columns, axis=1, inplace=True, errors='ignore')
    self.df_test.drop(columns, axis=1, inplace=True, errors='ignore')

    self.df_train['location_id']  = pd.to_numeric(self.df_train['location_id'], downcast='integer')
    self.df_val['location_id']  = pd.to_numeric(self.df_val['location_id'], downcast='integer')
    self.df_test['location_id']  = pd.to_numeric(self.df_test['location_id'], downcast='integer')

ModelHelper.drop_all_but_target = drop_all_but_target

In [ ]:
class WindowGenerator:

  def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df, label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [ ]:
def split_window(self, features):

  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [ ]:
def make_dataset(self, data, sequence_stride=1):

  data = np.array(data, dtype=np.float32)
  ds = timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=sequence_stride,
      shuffle=False,
      batch_size=self.batch_size,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
def set_window_generator(self, label_columns=None):

    window_generator = WindowGenerator(self.sequence_length, 1, 1, self.df_train, self.df_val, self.df_test, label_columns)
    window_generator.batch_size = self.batch_size
    self.window_generator = window_generator

ModelHelper.set_window_generator = set_window_generator

In [ ]:
def make_windowed_dataset(self, sequence_stride=1):

    self.train_dataset = self.window_generator.make_dataset(self.window_generator.train_df, sequence_stride)
    self.val_dataset = self.window_generator.make_dataset(self.window_generator.val_df, sequence_stride)
    self.test_dataset = self.window_generator.make_dataset(self.window_generator.test_df, sequence_stride)

ModelHelper.make_windowed_dataset = make_windowed_dataset

Model Assignment

In [ ]:
def assign_model(self, model):

    self.model = model

ModelHelper.assign_model = assign_model

In [ ]:
adam = tf.keras.optimizers.Adam(lr=0.001)

def compile_model(self, optimizer_type=tf.keras.optimizers.Adam, learning_rate=0.001):

    optimizer = optimizer_type(learning_rate)
    self.model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=tf.keras.metrics.SparseCategoricalAccuracy())

ModelHelper.compile_model = compile_model

In [ ]:
def fit_model(self, with_early_stopping=True, verbose="auto"):

    callbacks = []
    if with_early_stopping is True:
        callbacks.append(tf.keras.callbacks.EarlyStopping(monitor='val_sparse_categorical_accuracy', patience=3, restore_best_weights=True, verbose=1))

    self.model.fit(self.train_dataset,  validation_data=self.val_dataset, epochs=self.num_epochs, callbacks=callbacks, verbose=verbose)

ModelHelper.fit_model = fit_model

In [ ]:
def evaluate_model(self):

    self.model.evaluate(self.test_dataset)

ModelHelper.evaluate_model = evaluate_model

In [ ]:
def federated_training(self, create_model, optimizer=tf.keras.optimizers.Adam, client_lr=0.002, server_lr=0.06, tolerance=7, logging_path='./log/central-test-run'):

    train_logdir = logging_path + '/train'
    val_logdir = logging_path + '/val'

    train_summary_writer = tf.summary.create_file_writer(train_logdir)
    val_summary_writer = tf.summary.create_file_writer(val_logdir)

    # Clone the keras_model inside `create_tff_model()`, which TFF will
    # call to produce a new copy of the model inside the graph that it will
    # serialize. Note: we want to construct all the necessary objects we'll need
    # _inside_ this method.
    def create_tff_model():
        # TFF uses an `input_spec` so it knows the types and shapes
        # that your model expects.
        input_spec = self.federated_train_data[0].element_spec
        keras_model_clone = create_model()
        tff_model = tff.learning.from_keras_model(
          keras_model_clone,
          input_spec=input_spec,
          loss=tf.keras.losses.SparseCategoricalCrossentropy(),
          metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
        return tff_model


    # This command builds all the TensorFlow graphs and serializes them:
    fed_avg = tff.learning.build_federated_averaging_process(
    model_fn=create_tff_model,
    client_optimizer_fn=lambda: optimizer(learning_rate=client_lr),
    server_optimizer_fn=lambda: optimizer(learning_rate=server_lr))

    state = fed_avg.initialize()
    self.evaluation = tff.learning.build_federated_evaluation(model_fn=create_tff_model)

    best_state = 0
    lowest_loss = 100.00
    stop = tolerance

    with train_summary_writer.as_default():
        for round_num in range(1, self.num_epochs + 1):
          print('Round {r}'.format(r=round_num))

          # Uncomment to simulate sparse availability of clients
          # train_data_for_this_round, val_data_for_this_round = sample((federated_train_data, federated_val_data), 20, NUM_CLIENTS)

          state, metrics = fed_avg.next(state, self.federated_train_data)

          train_metrics = metrics['train']
          print('\tTrain: loss={l:.3f}, accuracy={a:.3f}'.format(l=train_metrics['loss'], a=train_metrics['sparse_categorical_accuracy']))

          val_metrics = self.evaluation(state.model, self.federated_val_data)
          print('\tValidation: loss={l:.3f}, accuracy={a:.3f}'.format( l=val_metrics['loss'], a=val_metrics['sparse_categorical_accuracy']))

          # Check for decreasing validation loss
          if lowest_loss > val_metrics['loss']:
            print('\tSaving best model..')
            lowest_loss = val_metrics['loss']
            best_state = state
            stop = tolerance - 1
          else:
            stop = stop - 1
            if stop <= 0:
              print('\tEarly stopping...')
              break;

          print(' ')
          print('\twriting..')

          # Iterate across the metrics and write their data
          for name, value in dict(train_metrics).items():
            tf.summary.scalar('epoch_'+name, value, step=round_num)

          with val_summary_writer.as_default():
            for name, value in dict(val_metrics).items():
              tf.summary.scalar('epoch_'+name, value, step=round_num)

    train_summary_writer.close()
    val_summary_writer.close()
    self.best_state = best_state

ModelHelper.federated_training = federated_training

In [ ]:
def federated_evaluation(self):

     # evaluate over test data
      test_metrics = self.evaluation(self.best_state.model, self.federated_test_data)
      print('\tEvaluation: loss={l:.3f}, accuracy={a:.3f}'.format( l=test_metrics['loss'], a=test_metrics['sparse_categorical_accuracy']))

ModelHelper.federated_evaluation = federated_evaluation

In [ ]:
def print_test_prediction_info(self):

    def print_info(arr, name):
      print(name)
      print('Shape : ', arr.shape)
      print('Example [0] : ', arr[0])
      if len(arr.shape) > 2:
        print('2nd axis example : ', arr[0][:][0])

    logits = self.model.predict(self.test_dataset)
    print_info(logits, 'logits')

    predictions = tf.nn.softmax(logits, axis=1)
    print_info(predictions, 'predictions')

    predicted_classes = np.argmax(predictions, 1)
    print_info(predicted_classes, 'predicted_classes')

    actual_values = []
    for x, y in self.test_dataset.unbatch():
        actual_values.append(y.numpy())

    actual_values = np.array(actual_values)
    print_info(actual_values, 'actual_values')

    diff = actual_values - predicted_classes
    print_info(diff, 'diff')

    for i in range(0,20):
        print('Prediction #', i)
        print('Actual values: ', actual_values[i])
        print('Predicted values: ', predicted_classes[i])

    wrong = np.count_nonzero(diff)
    size = diff.shape[0]
    correct =  size - wrong
    acc = correct / size
    print('# correct Predictions : ', correct)
    print('# wrong Predictions : ', wrong)
    print('accuracy: ', acc)

ModelHelper.print_test_prediction_info = print_test_prediction_info

legacy function to fix train, val and test datasets

In [ ]:
def fix_length_dfs(self):

    def fix_length_df(df, batch_size):
        if df.shape[0] % batch_size == 0:
            return df

        idx = -1 * (df.shape[0] % batch_size)
        print(idx)
        return df[:idx]

    fix_length_df(self.df_train, self.batch_size)
    fix_length_df(self.df_val, self.batch_size)
    fix_length_df(self.df_test, self.batch_size)

In [ ]:
def create_users_locations_from_df(self):
    users_locations = []

    # For each user
    for c_id in tqdm(self.get_unique_clients()):
        p = self.df.loc[self.df[self.client_column] == c_id].copy()
        users_locations.append(p)

    self.users_locations = users_locations
    return users_locations


ModelHelper.create_users_locations_from_df = create_users_locations_from_df

In [ ]:
def concat_split_users_locations(self, drop_client_column=True):

    users_locations_train = []
    users_locations_val = []
    users_locations_test = []

    for user_df in self.users_locations:
        # Split in train, test and validation
        train, test = train_test_split(user_df, test_size=0.2, shuffle=False)
        train, val = train_test_split(train, test_size=0.2, shuffle=False)

        # Append the sets
        users_locations_train.append(train)
        users_locations_val.append(val)
        users_locations_test.append(test)

    # Merge back the dataframes
    df_train = pd.concat(users_locations_train)
    df_val = pd.concat(users_locations_val)
    df_test = pd.concat(users_locations_test)

    if drop_client_column:
        df_train.drop([self.client_column], axis=1, inplace=True)
        df_val.drop([self.client_column], axis=1, inplace=True)
        df_test.drop([self.client_column], axis=1, inplace=True)

    #df_train['location_id']  = pd.to_numeric(df_train['location_id'], downcast='integer')
    #df_val['location_id']  = pd.to_numeric(df_val['location_id'], downcast='integer')
    #df_test['location_id']  = pd.to_numeric(df_test['location_id'], downcast='integer')

    self.df_train = df_train
    self.df_val = df_val
    self.df_test = df_test

ModelHelper.concat_split_users_locations = concat_split_users_locations

In [ ]:
def create_tuner(self, create_keras_model, compile_keras_model, max_epochs=10, factor=3):

    def create_and_compile(hp):
        model=create_keras_model(hp)
        compile_keras_model(hp, model)

        return model

    self.tuner = kt.Hyperband(create_and_compile,
                     objective='val_sparse_categorical_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='keras_tuner',
                     project_name='mobility_prediction')

ModelHelper.create_tuner = create_tuner

In [ ]:
def tuner_search(self, num_epochs=20, stop_early=True, num_trials=1):
    callbacks =[]
    if stop_early:
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)]

    self.tuner.search(self.train_dataset, validation_data=self.val_dataset, epochs=num_epochs, callbacks=callbacks)

    self.best_hps = self.tuner.get_best_hyperparameters(num_trials=num_trials)[0]

ModelHelper.tuner_search = tuner_search

In [ ]:
def tuner_find_best_epoch(self, num_epochs=20):
    self.model = self.tuner.hypermodel.build(self.best_hps)
    history = self.model.fit(self.train_dataset, validation_data=self.val_dataset, epochs=num_epochs)

    val_acc_per_epoch = history.history['val_sparse_categorical_accuracy']
    self.best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1

ModelHelper.tuner_find_best_epoch = tuner_find_best_epoch

In [ ]:
def tuner_eval_model(self):
    self.model = self.tuner.hypermodel.build(self.best_hps)
    # Retrain the model
    self.model.fit(self.train_dataset, validation_data=self.val_dataset, epochs=self.best_epoch)
    self.model.evaluate(self.test_dataset)

ModelHelper.tuner_eval_model = tuner_eval_model

In [42]:
def create_tuner(self, create_keras_model, compile_keras_model, max_epochs=10, factor=3):

    def create_and_compile(hp):
        model=create_keras_model(hp)
        compile_keras_model(hp, model)

        return model

    self.tuner = kt.Hyperband(create_and_compile,
                     objective='val_sparse_categorical_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='keras_tuner',
                     project_name='mobility_prediction')

ModelHelper.create_tuner = create_tuner

In [43]:
def tuner_search(self, num_epochs=20, stop_early=True, num_trials=1):
    callbacks =[]
    if stop_early:
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1)]

    self.tuner.search(self.train_dataset, validation_data=self.val_dataset, epochs=num_epochs, callbacks=callbacks)

    self.best_hps = self.tuner.get_best_hyperparameters(num_trials=num_trials)[0]

ModelHelper.tuner_search = tuner_search

In [44]:
def tuner_find_best_epoch(self, num_epochs=20):
    self.model = self.tuner.hypermodel.build(self.best_hps)
    history = self.model.fit(self.train_dataset, validation_data=self.val_dataset, epochs=num_epochs)

    val_acc_per_epoch = history.history['val_sparse_categorical_accuracy']
    self.best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1

ModelHelper.tuner_find_best_epoch = tuner_find_best_epoch

In [45]:
def tuner_eval_model(self):
    self.model = self.tuner.hypermodel.build(self.best_hps)
    # Retrain the model
    self.model.fit(self.train_dataset, validation_data=self.val_dataset, epochs=self.best_epoch)
    self.model.evaluate(self.test_dataset)

ModelHelper.tuner_eval_model = tuner_eval_model